In [2]:
import pandas as pd
import numpy as np

In [3]:
pl0 = pd.read_csv("Kraftwerksliste_CSV.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=9)
bm = pd.read_csv("block_plant_mapper.csv", error_bad_lines=False, engine="python", sep=";")

In [4]:
pl0.rename(columns={ pl0.columns[10]: "Energieträger_old", pl0.columns[13]: "Energieträger"}, inplace=True)

In [5]:
#bm

In [6]:
#bm
pl1 = pl0.loc[np.logical_or(pl0.Energieträger == "Kernenergie", np.logical_or(pl0.Energieträger == "Erdgas", np.logical_or(pl0.Energieträger == "Steinkohle", pl0.Energieträger == "Braunkohle"))), :]

In [75]:
pl2 = pl1.rename(columns={ pl1.columns[0]: "BlockID", pl1.columns[3]: "PLZ", pl1.columns[4]: "Ort", pl1.columns[8]: "Inbetriebnahme", pl1.columns[9]: "Status", pl1.columns[15]: "KWK", pl1.columns[16]: "Nettoleistung"})

In [76]:
cols = [10, 11,12,14,17,18,19]
pl3 = pl2.drop(pl2.columns[cols],axis=1)

In [101]:
#pl3

In [143]:
#gas = pl3.loc[pl1.Energieträger == "Steinkohle"]

In [205]:
gases = pl3.loc[:]

In [206]:
atest = gases.groupby(["PLZ"]).count()

In [296]:
plz = "24939"
gases.loc[gases.PLZ == plz]

,BlockID,Unternehmen,Kraftwerksname,PLZ,Ort,Straße und Hausnummer (Standort Kraftwerk),Bundesland,Blockname,Inbetriebnahme,Status,Energieträger,KWK,Nettoleistung
436,BNA1819,Stadtwerke Flensburg GmbH,Heizkraftwerk FL,24939,Flensburg,Batteriestr. 48,Schleswig-Holstein,Block 12,2016,in Betrieb,Erdgas,Ja,75
437,BNA0270,Stadtwerke Flensburg GmbH,Heizkraftwerk FL,24939,Flensburg,Batteriestr. 48,Schleswig-Holstein,Block 11,1992,in Betrieb,Steinkohle,ja,27
438,BNA0271,Stadtwerke Flensburg GmbH,Heizkraftwerk FL,24939,Flensburg,Batteriestr. 48,Schleswig-Holstein,Block 10,1988,in Betrieb,Steinkohle,ja,29
439,BNA0272,Stadtwerke Flensburg GmbH,Heizkraftwerk FL,24939,Flensburg,Batteriestr. 48,Schleswig-Holstein,Block 9,1985,in Betrieb,Steinkohle,ja,33
2004,BNA0273,Stadtwerke Flensburg GmbH,Heizkraftwerk FL,24939,Flensburg,Batteriestr. 48,Schleswig-Holstein,Block 8,1982,Endgültig Stillgelegt 2016 (mit StA),Steinkohle,ja,35
2005,BNA0274,Stadtwerke Flensburg GmbH,Heizkraftwerk FL,24939,Flensburg,Batteriestr. 48,Schleswig-Holstein,Block 7,1978,Endgültig Stillgelegt 2016 (mit StA),Steinkohle,ja,23


In [8]:
pl2 = pl1.rename(columns={ pl1.columns[0]: "BlockID", pl1.columns[3]: "PLZ", pl1.columns[4]: "Ort", pl1.columns[8]: "Inbetriebnahme", pl1.columns[9]: "Status", pl1.columns[15]: "KWK", pl1.columns[16]: "Nettoleistung"})
#pl3 = pl2.rename(columns={ pl1.columns[1]: "BlockID" })

In [9]:
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('.','')
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(',','.')
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('\n','')
#pl2['Nettoleistung'] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(';','.')

In [10]:
#pl2[] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
pl2['Nettoleistung'] = pl2['Nettoleistung'].apply(pd.to_numeric)

In [11]:
# pl2

In [12]:
cols = [10, 11,12,14,17,18,19]
pl3 = pl2.drop(pl2.columns[cols],axis=1)

In [13]:
# pl3

In [14]:
# Status: endgültig stillgelegt, vorläufig stillgelegt, in Betrieb, Sicherheitsbereitschaft, Sonderfall, 
# Stillegung: [Jahreszahl]
# StA: ja/ nein

In [15]:
newdf = pd.DataFrame()

In [16]:
def generate_decom_tuple(state):
    if state == "In Betrieb":
        return("in Betrieb", "0")
    elif "Endgültig Stillgelegt" in state:
        a = "stillgelegt"
        b = str(int(state[22:26]))
        return (a, b)
    elif "Vorläufig Stillgelegt" in state:
        a = "vorläufig stillgelegt"
        return (a, "0")
    else:
        return(state, "0")

In [17]:
def fix_company(company):
    cmp = company
    company_prefix, _ = company.split(" " , 1)
    # company_dict = {}
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH", "Nordzucker": "Nordzucker AG"}
    try:
        cmp = company_dict[company_prefix]
    except KeyError:
        return company
    return cmp

In [18]:
for index, row in pl3.iterrows():
    #if row['Status'] == "Endgültig Stillgelegt 2016 (mit StA)":
    # print(row)
    if row['KWK'] == "nein":
        row['KWK'] = "Nein" # workaround for BoA plants
    status, stilllegung = generate_decom_tuple(row['Status'])
    cmp = fix_company(row['Unternehmen'])
    row['Unternehmen'] = cmp
    row['Status'] = status
    row['Stilllegung'] = stilllegung
    newdf = newdf.append(row)
    

In [19]:
# newdf['Stilllegung'] = newdf['Stilllegung'].astype(int)
#newdf['Stilllegung'] = newdf['Stilllegung'].apply(lambda x: int(x) if x == x else None)

In [20]:
newdf['Stilllegung'] = newdf['Stilllegung'].replace("0", np.nan)

In [21]:
pl3 = newdf

In [22]:
atest = newdf.groupby(["Unternehmen"]).count()
atest.sort_values("BlockID", inplace=True, ascending=False)

In [23]:
print(atest)

                                                    BlockID  Blockname  \
Unternehmen                                                              
RWE AG                                                   68         67   
Vattenfall GmbH                                          41         41   
Uniper SE                                                29         25   
EnBW AG                                                  22         22   
Steag GmbH                                               10         10   
SWM Services GmbH                                         9          9   
RheinEnergie AG                                           7          7   
Volkswagen AG                                             7          5   
Evonik Degussa GmbH                                       7          6   
Statkraft Markets GmbH                                    6          4   
Stadtwerke Flensburg GmbH                                 6          6   
K+S AG                                

In [24]:
atest = pl3.groupby(["Status"]).count()
# atest

In [25]:
# pl3.rename(columns={ pl3.columns[10]: "Energieträger"}, inplace=True)

In [26]:
pl4 = bm.merge(pl3, how="outer", on="BlockID")

In [27]:
pl5 = pl4[pd.notnull(pl4['BlockID'])]
#pl5 = pl4
pl6 = pl5[pd.notnull(pl5['Nettoleistung'])]

In [28]:
# pl6

In [29]:
blocks = pl6.copy()
stammdaten = pl6.copy()

In [30]:
blocks.sort_values("Nettoleistung", inplace=True, ascending=False)
# blocks

In [31]:
plants = blocks.loc[:]
plants = plants.dropna(subset=["KraftwerkID"])

In [32]:
plants_d = plants.loc[plants.duplicated("KraftwerkID", keep=False)]
# plants_d

In [33]:
plants_c = plants_d.groupby(["KraftwerkID"]).sum()
plants_count = plants_d.groupby(["KraftwerkID"]).count()

In [34]:
plants_f = plants_c.reset_index()

In [35]:
plants_count = plants_count.loc[:, ["BlockID"]]

In [36]:
plants_count = plants_count.reset_index()
# plants_count 

In [37]:
plants_f = plants_f.merge(plants_count, on="KraftwerkID", how="inner")
# plants_f

In [38]:
plants_a = plants.drop_duplicates(subset="KraftwerkID")

In [39]:
plants_b = plants_d.loc[:]

In [40]:
plants_f = plants_f.rename(columns={ "Nettoleistung": "Gesamtleistung", "BlockID": "Blockzahl"})

In [41]:
plants_f = plants_f.merge(plants_a, on="KraftwerkID", how="left")
# plants_f

In [42]:
blocks.drop_duplicates(subset="BlockID", inplace=True)

In [43]:
#cols = [4,5,6,8]
#blocks.drop(blocks.columns[cols],axis=1, inplace=True)
#test_blocks = blocks.drop(blocks.columns[cols],axis=1)

In [44]:
blocks.drop(['Ort', 'PLZ', 'Straße und Hausnummer (Standort Kraftwerk)', 'Blockname'], axis=1, inplace=True)

In [45]:
plants_f.sort_values("Gesamtleistung", ascending=False, inplace=True)

In [46]:
drop_list = ['Blockname', 'KWK', 'Nettoleistung', 'Stilllegung', 'Straße und Hausnummer (Standort Kraftwerk)', 'PLZ', 'Ort', 'Nettoleistung']
plants_g = plants_f.drop(drop_list, axis=1)

In [47]:
plants_h = plants_g.rename(columns={"KraftwerkID": "plantid", "Gesamtleistung": "totalpower", "Blockzahl": "blockcount", "BlockID": "blockid", "Bundesland": "federalstate", "Energieträger": "energysource", "Inbetriebnahme": "latestexpanded", "Kraftwerksname": "plantname", "Status": "state", "Unternehmen": "company"})

In [48]:
plants_h['latestexpanded'] = plants_h['latestexpanded'].astype(int)

In [49]:
plants_i = plants_h[["plantid", "plantname", "federalstate", "energysource", "latestexpanded", "totalpower", "state", "blockcount", "blockid", "company"]]

In [50]:
plants_h.shape

(35, 10)

In [51]:
plants_i.shape

(35, 10)

In [52]:
plants_h.dtypes

plantid            object
totalpower        float64
blockcount          int64
blockid            object
federalstate       object
energysource       object
latestexpanded      int64
plantname          object
state              object
company            object
dtype: object

In [53]:
# plants_i

In [54]:
# test_blocks

In [55]:
#blocks.loc[blocks.Energieträger == 'Erdgas',]

In [56]:
plants_final = plants_i.loc[:]

In [57]:
# plants_final

In [58]:
# stammdaten

In [59]:
# cols = [0,2,3,9,10,11,12]
drop_list = ['KraftwerkID', 'Blockname', 'Energieträger', 'Inbetriebnahme', 'KWK', 'Kraftwerksname', 'Nettoleistung', 'Status', 'Stilllegung', 'Unternehmen']
stammdaten.drop(drop_list, axis=1, inplace=True)

In [60]:
#stammdaten.to_csv("stammdaten.csv", index=False)

In [61]:
#plants_final.to_csv("plants.csv", index=False)

In [62]:
#blocks.to_csv("blocks_3.csv", index=False)

In [63]:
#stammdaten.to_csv("stammdaten_nh.csv", index=False, header=False)
#blocks.to_csv("blocks_nh.csv", index=False, header=False)
#plants_final.to_csv("plants_nh.csv", index=False, header=False)